#Import Libraries

In [1]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 8.5 MB/s 


In [2]:
import numpy as np 
import os
import PIL
import pandas as pd
from google.colab.patches import cv2_imshow
import cv2
import math
import pickle
import tensorflow as tf
import urllib.request
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,TimeDistributed,Dense,LSTM,Flatten,Input
from tqdm import tqdm
from tensorflow_addons.optimizers import AdamW

# Dataset

##Download

In [20]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [21]:
!kaggle datasets download -d sreevishnukumarv/video-classification-datasetsubset-of-ucf101

100% 1.04G/1.04G [00:10<00:00, 120MB/s]
100% 1.04G/1.04G [00:10<00:00, 103MB/s]


In [22]:
!unzip /content/video-classification-datasetsubset-of-ucf101.zip

Archive:  /content/video-classification-datasetsubset-of-ucf101.zip
  inflating: test.csv                
  inflating: test/v_ApplyEyeMakeup_g01_c01.avi  
  inflating: test/v_ApplyEyeMakeup_g01_c02.avi  
  inflating: test/v_ApplyEyeMakeup_g01_c03.avi  
  inflating: test/v_ApplyEyeMakeup_g01_c04.avi  
  inflating: test/v_ApplyEyeMakeup_g01_c05.avi  
  inflating: test/v_ApplyEyeMakeup_g01_c06.avi  
  inflating: test/v_ApplyEyeMakeup_g02_c01.avi  
  inflating: test/v_ApplyEyeMakeup_g02_c02.avi  
  inflating: test/v_ApplyEyeMakeup_g02_c03.avi  
  inflating: test/v_ApplyEyeMakeup_g02_c04.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c01.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c02.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c03.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c04.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c05.avi  
  inflating: test/v_ApplyEyeMakeup_g03_c06.avi  
  inflating: test/v_ApplyEyeMakeup_g04_c01.avi  
  inflating: test/v_ApplyEyeMakeup_g04_c02.avi  
  inflating:

## Pre-Processing

In [23]:
def get_the_centered_frame(imgs_array,new_highet,new_width):
   img = cv2.resize(imgs_array[len(imgs_array)//2],(new_highet,new_width))
   return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [24]:
def compress_video(video_array,number_of_frames,new_highet,new_width):
  new_video = np.zeros((number_of_frames,new_highet,new_width))
  offset = video_array.shape[0] // number_of_frames
  for frame_index in range(number_of_frames):
    start = frame_index*offset
    end = start + offset
    if frame_index == number_of_frames - 1:
      end += video_array.shape[0] % number_of_frames
    new_video[frame_index] = get_the_centered_frame(video_array[start:end,:,:,:],new_highet,new_width)
  return new_video

In [25]:
def convert_avi_to_ndarray(filename):
	vidcap = cv2.VideoCapture(filename)
	video_list = []
	success,image = vidcap.read()
	while success:
		video_list.append(image)    
		success,image = vidcap.read()
	return np.array(video_list)

In [26]:
def uniform_video_frames(video,orignal_video_frames,current_video_frames,new_highet,new_width,CONSTANT_VIDEO_FRAMES = 75):
	while (current_video_frames < CONSTANT_VIDEO_FRAMES):
		if (math.floor((CONSTANT_VIDEO_FRAMES - current_video_frames) / orignal_video_frames) > 0):
			padding = orignal_video_frames
		else:
			padding = (CONSTANT_VIDEO_FRAMES - current_video_frames) % orignal_video_frames
		new_video_clip = video[:padding,:,:,:]
		video = np.concatenate([video,new_video_clip])
		current_video_frames+=padding
	new_video = []
	for frame in video:
		img = cv2.resize(frame,(new_highet,new_width))
		img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		new_video.append(img)
	return np.array(new_video)

In [27]:
def make_X_and_y(path,number_of_frames,new_highet,new_width,video_name_column='video_name',video_class_column='tag'):
  X = []
  y = []
  df = pd.read_csv(path + '.csv')
  for _,row in df.iterrows():
    video_name = row[video_name_column]
    video_array = convert_avi_to_ndarray(path + '/' + video_name)
    orignal_video_frames = video_array.shape[0]
    if orignal_video_frames < number_of_frames:
      X.append(uniform_video_frames(video_array,orignal_video_frames,orignal_video_frames,new_highet,new_width,CONSTANT_VIDEO_FRAMES = number_of_frames))
    else:
      X.append(compress_video(video_array,number_of_frames,new_highet,new_width))
    y.append(row[video_class_column])
  return X,y

In [28]:
X_train,y_train = make_X_and_y('/content/train',50,75,75)
X_test,y_test = make_X_and_y('/content/test',50,75,75)

In [29]:
X_train = np.array(X_train) / 255
X_test = np.array(X_test) / 255

In [30]:
lb = LabelEncoder()
y_cat_label_encoder = lb.fit_transform(y_train)
y_train_cat = to_categorical(y_cat_label_encoder)

lb = LabelEncoder()
y_cat_label_encoder = lb.fit_transform(y_test)
y_test_cat = to_categorical(y_cat_label_encoder)

##Save to Drive as TF Dataset

In [ ]:
def save_tf_dataset_in_drive(p,start_offset,end_offset,X,y):
  temp_train_dataset = tf.data.Dataset.from_tensor_slices((X[start_offset:end_offset],y[start_offset:end_offset]))
  tf.data.experimental.save(temp_train_dataset, p)

### Training

In [ ]:
training_samples_size = X_train.shape[0]

In [ ]:
number_of_training_chunks = 5
train_chunk_size = int(training_samples_size / number_of_training_chunks)

In [ ]:
p = '/content/drive/MyDrive/Kaggle/TF dataset/Batched dataset/Training/'
for i in range(number_of_training_chunks):
  save_tf_dataset_in_drive(p + '%i'%i,train_chunk_size*i,train_chunk_size*(i+1),X_train,y_train_cat)

### Testing

In [ ]:
testing_samples_size = X_test.shape[0]

In [ ]:
number_of_testing_chunks = 3
test_chunk_size = int(testing_samples_size / number_of_testing_chunks)

In [ ]:
p = '/content/drive/MyDrive/Kaggle/TF dataset/Batched dataset/Testing/'
for i in range(number_of_testing_chunks):
  save_tf_dataset_in_drive(p + '%i'%i,test_chunk_size*i,test_chunk_size*(i+1),X_test,y_test_cat)

# Neural Network

##Define the Model 

In [3]:
def inception_model_block():
  weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
  weights_file = "inception_v3.h5"
  urllib.request.urlretrieve(weights_url, weights_file)

  pre_trained_model = InceptionV3(input_shape=(75, 75, 1),include_top=False
                                  ,weights=None)
  
  for layer in pre_trained_model.layers:
    layer.trainable = False

  last_layer = pre_trained_model.layers[-1]
  x  = GlobalAveragePooling2D()(last_layer.output)
  
  return Model(pre_trained_model.input, x) 

In [4]:
def base_model(num_of_classes=15):
  inception_model = inception_model_block()
  video = Input(shape=(50, 75, 75, 1))
  frames_feature = TimeDistributed(inception_model)(video)
  x = LSTM(256,return_sequences = True)(frames_feature)
  x = Flatten()(x)
  x = Dense(250,activation='relu')(x)
  x = Dense(100,activation='relu')(x)
  predicted_class = Dense(num_of_classes,activation='softmax')(x)
  return Model(video, predicted_class) 

In [5]:
model = base_model()

In [6]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 75, 75, 1)]   0         
                                                                 
 time_distributed (TimeDistr  (None, 50, 2048)         21802208  
 ibuted)                                                         
                                                                 
 lstm (LSTM)                 (None, 50, 256)           2360320   
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 250)               3200250   
                                                                 
 dense_1 (Dense)             (None, 100)               25100     
                                                           

In [7]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [8]:
train_acc_metric = tf.keras.metrics.Accuracy()
val_acc_metric = tf.keras.metrics.Accuracy()

##Custom Training

### Train the Model

In [9]:
def apply_gradient(optimizer, model, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss_value = loss_object(y_true=y, y_pred=logits)
  
  gradients = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))
  
  return logits, loss_value

In [10]:
#I'll call this function 5 times (because I have 5 chunk of the training dataset) 
def train_data_for_one_epoch(train):
  losses = []
  pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  for step, (x_batch_train, y_batch_train) in enumerate(train):
      x_batch_train = tf.expand_dims(x_batch_train,axis = -1)
      logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
      
      losses.append(loss_value)
      
      train_acc_metric(y_batch_train, logits)
      pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
      pbar.update()
  return losses

In [11]:
def perform_validation(test):
  losses = []
  for x_val, y_val in test:
      val_logits = model(x_val)
      val_loss = loss_object(y_true=y_val, y_pred=val_logits)
      losses.append(val_loss)
      val_acc_metric(y_val, val_logits)
  return losses

In [ ]:
epochs = 10
epochs_val_losses, epochs_train_losses = [], []
p = '/content/drive/MyDrive/Kaggle/TF dataset/Batched dataset/'
train_path = p + 'Training/'
test_path = p + 'Testing/'
for epoch in range(epochs):
	print('Start of epoch %d' % (epoch,))
	train_acc = 0
	val_acc = 0
	for dir_name in os.listdir(train_path):
		train_dataset = tf.data.experimental.load(train_path + dir_name)
		batched_train_dataset = train_dataset.shuffle(buffer_size=16).batch(8)
		losses_train = train_data_for_one_epoch(batched_train_dataset)
		train_acc += train_acc_metric.result()
		del train_dataset,batched_train_dataset
		
	for dir_name in os.listdir(test_path):
		test_dataset = tf.data.experimental.load(test_path + dir_name)
		batched_test_dataset = test_dataset.shuffle(buffer_size=16).batch(8)
		losses_val = perform_validation(batched_test_dataset)
		val_acc += val_acc_metric.result()
		del test_dataset,batched_test_dataset

	losses_train_mean = np.mean(losses_train)
	losses_val_mean = np.mean(losses_val)
	epochs_val_losses.append(losses_val_mean)
	epochs_train_losses.append(losses_train_mean)
  
	print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))
	train_acc_metric.reset_states()
	val_acc_metric.reset_states()

### Test the Model

In [33]:
train_data_size = 1435

In [42]:
sample_index = np.random.randint(train_data_size)
video = np.reshape(X_train[sample_index],(1,50,75,75,1))

In [43]:
np.argmax(y_train_cat[sample_index])

9

In [44]:
y_pred = model.predict(video)

In [45]:
np.argmax(y_pred)

9

##Keras Training

In [31]:
X_train = np.expand_dims(X_train,-1) 
X_test = np.expand_dims(X_test,-1) 

In [ ]:
model = base_model()
opt_ = AdamW(learning_rate = 0.0001,weight_decay=1e-5)
model.compile(loss='categorical_crossentropy',optimizer=opt_,metrics=['accuracy'])
h = model.fit(X_train, y_train_cat,validation_data=(X_test,y_test_cat),epochs=5, batch_size=16,verbose=1)